In [1]:
# speed_tests.ipynb
# Authors: Stephan Meighen-Berger
# Testing different rotation implementations

<a href="https://colab.research.google.com/github/mjg-phys/cdm-computing-subgroup/blob/main/advancedPythonTutorial/nuisance/notebooks/speed_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [2]:
import numpy as np
import timeit

In [3]:
mixing_angles = np.array([
    [1, 2, np.arcsin(np.sqrt(0.312)) / np.pi * 180.],
    [1, 3, np.arcsin(np.sqrt(0.025)) / np.pi * 180., 0],  #  add cp violation here
    [2, 3, np.arcsin(np.sqrt(0.420)) / np.pi * 180.],
], dtype=object)

In [4]:
def rotmatrix_inefficient(dim: int, i: int, j: int, ang: float, cp: float):
    """ constructs a (Gell-Mann) rotational matrix ij with angle ang and cp violating phase cp
    for symmetry i < j is required

    Parameters
    ----------
    dim: int
        Dimensions of the matrix
    i, j: int
        Positions of the matrix
    ang: float
        Angle of roration in radians
    cp: float
        CP violating phase

    Returns
    -------
    rotmat: jax.numpy.array
        The rotation matrix

    Raises
    ------
    DimensionError:
        The input dimensions dim, i, j are wrong
    """
    # Building
    R = np.eye(dim, dtype=np.complex128)
    
    if cp == 0:
        for row in range(dim):
            for col in range(dim):
                if row == i and col == j:
                    R[row, col] = np.sin(ang)
                elif row == j and col == i:
                    R[row, col] = -np.sin(ang)
                elif row == col == i or row == col == j:
                    R[row, col] = np.cos(ang)
                else:
                    R[row, col] = 0.0
    else:
        for row in range(dim):
            for col in range(dim):
                if row == i and col == j:
                    R[row, col] = np.sin(ang) * np.exp(-1j * cp)
                elif row == j and col == i:
                    R[row, col] = -np.sin(ang) * np.exp(1j * cp)
                elif row == col == i or row == col == j:
                    R[row, col] = np.cos(ang)
                else:
                    R[row, col] = 0.0
    return R

def buildmixingmatrix_inefficient(params: np.ndarray) -> np.ndarray:
    """ constructs the mixing matrix from the input parameters
    for symmetry i < j is required

    For CP-violating factors, use tuples like (i,j,theta_ij,delta_ij),
    with delta_ij in degrees.

    Parameters
    ----------
    params: jax.numpy.array
        tuples descriping the mixing matrix.
        The format of each tuple should be [i, j, theta_ij]

    Returns
    -------
    mixing_matrix: jax.numpy.array
        The mixing matrix constructed in revers order.
        E.g. params = [(1,2,33.89),(1,3,9.12),(2,3,45.00)]
        => U = R_23 . R_13 . R_12
    """
    dim = max(np.array([par[1] for par in params]))
    U = np.eye(dim)
    # Applying the rotation matrices
    for par in params:
        if len(par) > 3:
            U = np.dot(rotmatrix_inefficient(dim, par[0] - 1, par[1] - 1, np.deg2rad(par[2]), np.deg2rad(par[3])), U)
        else:
            U = np.dot(rotmatrix_inefficient(dim, par[0] - 1, par[1] - 1, np.deg2rad(par[2]) , 0), U)
    return U

In [5]:
# Chat GPT rewrote it to be wrong!
def testing_inefficient():
    buildmixingmatrix_inefficient(mixing_angles)
testing_inefficient()

In [6]:
print(timeit.timeit("testing_inefficient()", globals=locals(), number=1000000))
print(timeit.timeit("testing_inefficient()", globals=locals(), number=1))

21.644977496005595
5.574498209170997e-05
